In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from scipy.integrate import quad
#Central density
rhoc = 1.0e17
#Constants
G = 6.674e-11
R = 1.19e4
c = 3e8
#Harmonics
l1 = 2
beta1 = l1*(l1+1)
#Range of the radial coordinate
r_min = 1e-12
r_max = R
n_points = 1000
r = np.linspace(r_min, r_max, n_points)
#Background density
def rhof(r): 
	x = (r/R)
	rhof = rhoc*(np.sinc(x))
	return rhof

#Background pressure
def pf(r):
    x = (r/R)
    pf = ((2*G*(rhoc**2)*(R**2))/np.pi)*((np.sinc(x))**2)
    return pf

#Square of the speed of sound
def csf(r): 
	x = (r/R)
	csf = 10*G*rhoc*(R**2)*((1)/(3*np.pi))*(np.sinc(x))
	return csf
	
#Gravitational acceleration
def gf(r): 
	x = (r/R)
	gf = 4*G*R*rhoc*((np.cos(x)/x) - (np.sinc(x)/x))
	return gf
	
#Gradient of background density
def rhogradf(r): 
	x = (r/R)
	rhogradf = rhoc*(np.pi/R)*((np.cos(x)/x) - (np.sinc(x)/x))
	return rhogradf

#Gradient of square of the speed of sound
def csgradf(r): 
	x = (r/R)
	csgradf = (10/3)*G*rhoc*R*((np.cos(x)/x) - (np.sinc(x)/x))
	return csgradf
	
rho = rhof(r)
cs = csf(r)
g = gf(r)
rhograd = rhogradf(r)
csgrad = csgradf(r)
#Adiabatic factor
Gamma = 2.2
Lambda1 = 3.2
Lambda2 = 4.2	
#The coupled ODEs
def eigenvalue_problem(r, y, om2):
	DPhi = y[0]
	Phi = y[1]
	W = y[2]
	V = y[3]
	dDPhidr = -(2/r)*DPhi + (((beta1/r)**2) - ((4*np.pi*G*rhof(r))/csf(r)))*Phi -	4*np.pi*G*(rhogradf(r) + ((rhof(r)*gf(r))/(csf(r))))*W + ((4*np.pi*G*rhof(r)*(om2[0])*r)/csf(r))*V
	dPhidr = DPhi
	dWdr = (1/(csf(r)))*Phi + ((gf(r)/csf(r)) - (2/r))*W + (((beta1**2)/r) - (((om2[0])*r)/(csf(r))))*V
	dVdr = ((gf(r)/(csf(r)*(om2[0])*r)) + ((rhogradf(r))/(rhof(r)*(om2[0])*r)))*Phi + ((1/r) + ((Lambda2*gf(r)*rhogradf(r))/((om2[0])*r*rhof(r))) + (((gf(r)**2)*Lambda1)/(csf(r)*r*(om2[0]))) + ((rhogradf(r)*csgradf(r))/(rhof(r)*r*(om2[0]))) + ((csf(r)*(rhogradf(r)**2))/((rhof(r)**2)*(om2[0])*r)) + ((gf(r)*csgradf(r))/(csf(r)*r*(om2[0]))))*W -  (((gf(r))/(csf(r))) + (1/r) + (rhogradf(r)/rhof(r)))*V
	return np.vstack((dDPhidr, dPhidr, dWdr, dVdr))
	
#boundary conditions
def bc(y0, yR, om2):
	R = 1.19e4
	return np.array([y0[0], y0[1], y0[2], yR[0] + (3/R)*yR[1], yR[1] - om2[0]*R*yR[3]])

#initial solution guess
om2 = np.zeros(1)
y_guess = np.vstack((-3*r, r**2, r**2, r))
om2[0] = 0.1225
#Solution of the BVP
res_first = solve_bvp(eigenvalue_problem, bc, r, y_guess, om2)
#1st g mode
gom1 = res_first.p
y_guess = res_first.y
#Solution of the BVP, 2nd g-mode
res_second = solve_bvp(eigenvalue_problem, bc, r, y_guess, gom1)
#2nd g mode
gom2 = res_second.p
print("The first eigenfrequency is: ", np.sqrt(gom1))
print("The second eigenfrequency is: ", np.sqrt(gom2))
#Calculate the perturbed density
def Drho1f(r): 
	Drho1f = -rhof(r)*((res_first.y[1]/csf(r)) + ((gf(r)*res_first.y[2])/csf(r)) - ((gom1*r*res_first.y[3])/csf(r)) + ((res_first.y[2]*rhogradf(r))/rhof(r)))
	return Drho1f

def Drho2f(r): 
	Drho2f = -rhof(r)*((res_second.y[1]/csf(r)) + ((gf(r)*res_second.y[2])/csf(r)) - ((gom2*r*res_second.y[3])/csf(r)) + ((res_second.y[2]*rhogradf(r))/rhof(r)))
	return Drho2f

def Drho1(r): 
	Drho1 = Drho1f(r)*(r**4)
	return Drho1

def Drho2(r): 
	Drho2 = Drho2f(r)*(r**4)
	return Drho2



The first eigenfrequency is:  [0.34999995]
The second eigenfrequency is:  [0.35018847]


In [18]:
display(res_first)

       message: The maximum number of mesh nodes is exceeded.
       success: False
        status: 1
             x: [ 1.000e-12  1.191e+01 ...  1.189e+04  1.190e+04]
           sol: <scipy.interpolate._interpolate.PPoly object at 0x11fc2dd90>
             p: [ 1.225e-01]
             y: [[ 2.026e-21  1.991e+03 ... -7.277e+13  4.330e+13]
                 [-2.831e-22  4.309e+03 ... -1.716e+17 -1.718e+17]
                 [ 1.847e-07  1.847e+02 ... -8.889e+07 -4.705e-03]
                 [ 6.060e-09 -4.999e+04 ... -1.448e+14 -1.178e+14]]
            yp: [[-1.019e+04 -9.552e+03 ...  2.306e+12 -5.095e+10]
                 [ 2.026e-21  1.991e+03 ... -7.277e+13  4.330e+13]
                 [-1.513e+05 -1.511e+05 ... -4.384e+11  1.049e+12]
                 [ 1.787e+05  1.535e+05 ... -1.598e+16  2.861e+20]]
 rms_residuals: [ 1.453e+00  1.425e+00 ...  1.279e+00  1.588e+00]
         niter: 1

In [ ]:
Multipole moments
#I1, error = quad(Drho1(r), r_min, R)
#I2, error = quad(Drho2(r), r_min, R)
#Orbital velocity
#OM_min = 0.001
#OM_max = 0.1
##OM = np.linspace(OM_min, OM_max, n_points)
#Love numbers
#def k1(OM): 
#	k1 = ((2*np.pi*G)/(5*(R**5)))*(I1**2)/((gom1**2) - (2*(OM**2)))
	return k1

def k2(OM): 
	k2 = ((2*np.pi*G)/(5*(R**5)))*(I2**2)/((gom2**2) - (2*(OM**2)))
	return k2

def kf(OM): 
	kf = k1(OM) + k2(OM)
	return kf

k = kf(OM)
plt.plot(OM, k)
plt.show()